# Probabilistic Programming Module


Here, we discuss how to use the probabilistic programming module to synthetically generate CDMs.



In [1]:
import kessler
import pyro
import numpy as np
import dsgp4

In [2]:
#we seed everything for reproducibility
pyro.set_rng_seed(10)

#we define the observing instruments
#GNSS first:
gnss_cov_rtn=np.array([1e-9, 1.115849341564346, 0.059309835843067, 1e-9, 1e-9, 1e-9])**2,
instrument_characteristics_gnss={'bias_xyz': np.array([[0., 0., 0.],
       [0., 0., 0.]]), 'covariance_rtn': gnss_cov_rtn}
gnss = kessler.GNSS(instrument_characteristics_gnss)

#and then radar:
radar_cov_rtn=np.array([1.9628939405514678, 2.2307686944695706, 0.9660907831563862, 1e-9, 1e-9, 1e-9])**2
instrument_characteristics_radar={'bias_xyz': np.array([[0., 0., 0.],
       [0., 0., 0.]]), 'covariance_rtn': radar_cov_rtn}
radar = kessler.Radar(instrument_characteristics_radar)


In [3]:
tles=dsgp4.tle.load('tles_sample_population.txt')
print(f"loaded {len(tles)} TLEs")

loaded 20 TLEs


In [6]:
# tles=dsgp4.tle.load('tles_sample_population.txt')

# tles_filtered=[]
# for tle in tqdm(tles):
#     try: 
#         dsgp4.initialize_tle(tle)
#         if tle.apogee_alt()<560*1e3:
#             tles_filtered.append(tle)
#     except:
#         continue

In [5]:
conjunction_model = kessler.model.ConjunctionSimplified(time0=60727.13899462018,
                                                        # max_duration_days=7.0,
                                                        # time_resolution=600000.0,
                                                        # time_upsample_factor=100,
                                                        miss_dist_threshold=5000.0,
                                                        prior_dict=None,
                                                        t_prob_new_obs=0.96,
                                                        c_prob_new_obs=0.4,
                                                        cdm_update_every_hours=8.0,
                                                        mc_samples=100,
                                                        mc_upsample_factor=100,
                                                        pc_method='MC',
                                                        collision_threshold=70,
                                                        # likelihood_t_stddev=[371.068006, 0.0999999999, 0.172560879],
                                                        # likelihood_c_stddev=[371.068006, 0.0999999999, 0.172560879],
                                                        likelihood_time_to_tca_stddev=0.7,
                                                        t_observing_instruments=[gnss],
                                                        c_observing_instruments=[radar],
                                                        tles=tles,)

In [6]:
trace,iterations=conjunction_model.get_conjunction()

/Users/ga00693/Develop/kessler/kessler/model.py:765: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pyro.deterministic('conj',torch.tensor(conj))
/Users/ga00693/Develop/kessler/kessler/util.py:85: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1738206012956/work/aten/src/ATen/native/Cross.cpp:66.)
  h_vec = torch.cross(r_vec, v_vec)


After 1170 iterations, generated event with 2 CDMs


In [9]:
trace.nodes['cdms']['infer']['cdms']

[CCSDS_CDM_VERS                        = 1.0
 CREATION_DATE                         = 2025-02-21T03:20:09.135184
 ORIGINATOR                            = KESSLER_SOFTWARE
 MESSAGE_ID                            = KESSLER_SOFTWARE_ac66059e-20e6-11f0-b2ee-f2a9f71f7e1a
 TCA                                   = 2025-02-22T01:17:28.408663
 MISS_DISTANCE                         = 141514.9960370336
 RELATIVE_SPEED                        = 8783.071096663718
 RELATIVE_POSITION_R                   = -3769.8761935982398
 RELATIVE_POSITION_T                   = -110791.09624826182
 RELATIVE_POSITION_N                   = -87963.71484285413
 RELATIVE_VELOCITY_R                   = -144.7709891581174
 RELATIVE_VELOCITY_T                   = -5424.100602146948
 RELATIVE_VELOCITY_N                   = -6906.555719570857
 COLLISION_PROBABILITY                 = 0.0
 COLLISION_PROBABILITY_METHOD          = MC
 OBJECT                                = OBJECT1
 OBJECT_DESIGNATOR                     = 76126AP

In [35]:
#let's save all cdms to kvn files:
#for i in range(len(trace.nodes['cdms']['infer']['cdms'])):    
#    trace.nodes['cdms']['infer']['cdms'][i].save(f'event2_{i}.kvn')